In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt

# Load datasets for omega and omega' (2w')
data_w16 = np.load('datasets_1002/D_dataset_0702_w16_v1.npz', allow_pickle=True)
data_w32 = np.load('datasets_1002/D_dataset_0702_w32_v1.npz', allow_pickle=True)

# Extract a single sample for testing
# u_true is the target solution we want to reach iteratively
f_phys = data_w32['src'][0] 
u_true = data_w32['wave'][0]

print(f"Physical Domain Shape: {f_phys.shape}")
print(f"Frequency Pair: omega_prime=16, omega=32")

In [ ]:
def iterative_refinement(L_w, L_w_prime, f, u_initial, T_op, iterations=5):
    u_k = u_initial.copy().flatten()
    f_vec = f.flatten()
    history = []
    
    for k in range(iterations):
        # 1. Calculate Residual at high frequency
        res = f_vec - L_w @ u_k
        res_norm = np.linalg.norm(res) / np.linalg.norm(f_vec)
        history.append(res_norm)
        
        # 2. Apply Transfer Operator T (Currently Identity)
        r_tilde = T_op(res)
        
        # 3. Solve at low frequency (the "Coarse" correction)
        # Using a direct solve here for clarity
        e_tilde = sla.spsolve(L_w_prime, r_tilde)
        
        # 4. Update the solution
        u_k = u_k + e_tilde
        
    return u_k.reshape(f.shape), history

# Placeholder for L matrices (requires build_finesse_solver from your source)
# L_32 = ... 
# L_16 = ...
# T_identity = lambda x: x